# Stock market clustering

_Data Structures and Algorithms_

_Imperial College Business School_


---
This assignment is divided into three parts. In the first part, you will work on `pandas` data analysis. In the second part, you will implement a clustering algorithm to group companies based on their stock price movements. In the final part, you will explore ways to extend and improve this analysis. 

---
- **Assessment criteria**
  - Graded on **both correctness and presentation**.
  - Make results easy to read: apply **string formatting**, include **plots** when useful, and **comment your code**.

- **Testing**
  - There are **no OK tests** for this assignment.
  - You are expected to **explore the data and problem**, and use a **search engine** to identify appropriate pandas functions.
  - See `veryUseful.py` for a short list of potentially helpful pandas functions.

- **Collaboration**
  - This is **group work**; consider dividing tasks.
  - Some team members can focus on the **pandas/data analysis** component, others on the **algorithmic** component.
  - **Intermediate results** are provided to **test your algorithm**; you can begin both parts immediately (see **Question 3** for details).

- **Use of generative AI**
  - **Permitted only in Part 3** of the assignment.
  - If used (or if other external sources are used) in Part 3, **clearly document how** you used them.
  - **Not allowed** in the other parts.
---


## Your group

You'll complete this assignment in your assigned study groups. If you are unsure about your group, please contact the programme team.

## Submission
Submit your work via GitHub, following the detailed instructions provided in the assessment submission guidelines.


## Part 1: Pandas

**30% of grade**

In the previous homework, we used lists to study stock prices. The `pandas` library provides some more effective tools for data analysis.

The assignment comes with two files containing company data:
- `SP_500_firms.csv` with firm and ticker names
- `SP_500_close_2015.csv` with stock price data for 2015

Let's first load up this data.

In [3]:
# Load data into Python

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv

def read_names_into_dict():
    """
    Read company names into a dictionary
    """
    d = dict()
    with open("SP_500_firms.csv") as csvfile:
        input_file = csv.DictReader(csvfile)
        for row in input_file:
            #print(row)
            d[row['Symbol']] = [row['Name'],row['Sector']]
    return d

names_dict = read_names_into_dict()
comp_names = names_dict.keys()

# Read price data with pandas
filename = 'SP_500_close_2015.csv'
price_data = pd.read_csv(filename, index_col=0)

### Question 1: Returns

In the previous homework, we calculated stock price _returns_ over a period of time. The return is defined as the percentage change, so the return between periods $t-1$ and $t$ for stock price $p$ would be

$$
x_t = \frac{p_t - p_{t-1}}{p_{t-1}}.
$$

Calculate the returns in `pandas` for all the stocks in `price_data`.

In [4]:
# Calculate company returns in this cell
data_unstack = pd.DataFrame(price_data.unstack()).reset_index().rename(columns={'level_0':'Stock',0:'Price'})
data_unstack['Date'] = pd.to_datetime(data_unstack['Date'])
data_unstack = data_unstack.sort_values(by=['Stock', 'Date'])
data_unstack['Return'] = data_unstack.groupby('Stock')['Price'].transform(lambda x: x / x.shift(1) - 1)

### Question 1.1: Highest and lowest daily returns

Use pandas to find the 10 highest daily returns amongst all companies. Search online for what were the reasons behind the highest returns. Present your results in a clean and immediately readable form.

Repeat with the lowest daily returns.

In [5]:
# Your code here
Top_10_highest_return = data_unstack.sort_values(by='Return', ascending=False).head(10)
Top_10_lowest_return = data_unstack.sort_values(by='Return', ascending=True).head(10)

In [6]:
Top_10_highest_return

,Stock,Date,Price,Return
49556,FCX,2015-08-27,10.150626,0.286616
121581,WMB,2015-06-22,54.483408,0.258999
111329,TRIP,2015-10-14,83.720001,0.255361
53946,HAR,2015-01-29,121.878258,0.237600
92698,QRVO,2015-11-06,55.549999,0.232254
122409,WYNN,2015-10-02,62.041515,0.228389
111160,TRIP,2015-02-12,82.400002,0.224915
57809,HUM,2015-05-29,212.955142,0.203128
64714,KLAC,2015-10-21,62.030356,0.187895
86753,PRGO,2015-04-08,193.748480,0.183899


In [7]:
Top_10_lowest_return

,Stock,Date,Price,Return
93187,PWR,2015-10-16,18.740000,-0.285006
74439,KORS,2015-05-27,45.930000,-0.241954
17528,BIIB,2015-07-24,300.029999,-0.220802
103776,SRCL,2015-10-23,120.309998,-0.192767
124176,YUM,2015-10-07,66.091086,-0.188324
88978,RL,2015-02-04,135.946571,-0.182169
74965,MU,2015-06-26,19.660000,-0.181515
82637,NRG,2015-12-04,8.831297,-0.179581
70523,MNK,2015-11-09,58.009998,-0.169981
3735,AKAM,2015-10-28,62.910000,-0.167306


### Question 1.2: Highest and lowest yearly returns

Find the 10 highest yearly returns amongst all companies. Present your results in a clean and immediately readable form.

Repeat with the lowest yearly returns.

In [8]:
# Your code here
# Calculate yearly return for each stock: (last price of year - first price of year) / first price of year
yearly_prices = data_unstack.set_index('Date').groupby('Stock').resample('Y')['Price'].agg(['first', 'last'])
yearly_prices['Yearly_Return'] = (yearly_prices['last'] - yearly_prices['first']) / yearly_prices['first']
Top_10_highest_yearly_return = yearly_prices.sort_values(by='Yearly_Return',ascending=False).head(10)
Top_10_lowest_yearly_return = yearly_prices.sort_values(by='Yearly_Return',ascending=True).head(10)

/tmp/ipykernel_2259/873161380.py:3: FutureWarning: 'Y' is deprecated and will be removed in a future version, please use 'YE' instead.
  yearly_prices = data_unstack.set_index('Date').groupby('Stock').resample('Y')['Price'].agg(['first', 'last'])


In [9]:
Top_10_highest_yearly_return

,,first,last,Yearly_Return
Stock,Date,,,
NFLX,2015-12-31,49.848572,114.379997,1.294549
AMZN,2015-12-31,308.519989,675.890015,1.190750
ATVI,2015-12-31,19.765196,38.397531,0.942684
AYI,2015-12-31,139.234407,233.418545,0.676443
NVDA,2015-12-31,19.642392,32.695044,0.664514
GPN,2015-12-31,40.276764,64.480414,0.600933
HRL,2015-12-31,25.076529,39.094699,0.559016
EXR,2015-12-31,55.660139,85.965350,0.544469
VRSN,2015-12-31,57.189999,87.360001,0.527540


In [10]:
Top_10_lowest_yearly_return

,,first,last,Yearly_Return
Stock,Date,,,
CHK,2015-12-31,19.546924,4.500000,-0.769785
SWN,2015-12-31,27.170000,7.110000,-0.738314
FCX,2015-12-31,22.855546,6.770000,-0.703792
KMI,2015-12-31,39.669553,14.597110,-0.632032
MU,2015-12-31,34.750000,14.160000,-0.592518
NRG,2015-12-31,26.202970,11.549374,-0.559234
RRC,2015-12-31,55.073725,24.570357,-0.553864
MRO,2015-12-31,27.352847,12.407730,-0.546383
MUR,2015-12-31,46.660241,21.531765,-0.538541


### Question 1.3: Highest and lowest volatilities

Find the 10 highest yearly volatilities (standard deviations) amongst all companies. Present your results in a clean and immediately readable form.

Repeat with the lowest volatilities.

In [11]:
# Your code here
yearly_volatilities = data_unstack.groupby('Stock').std().reset_index()[['Stock','Return']].rename(columns={'Return':'Volatilities'})
Top_10_highest_yearly_volatilities = yearly_volatilities.sort_values(by='Volatilities', ascending=False).head(10)
Top_10_lowest_yearly_volatilities = yearly_volatilities.sort_values(by='Volatilities', ascending=True).head(10)

In [12]:
Top_10_highest_yearly_volatilities

,Stock,Volatilities
174,FCX,0.044071
88,CHK,0.042784
384,RIG,0.037553
417,SWN,0.035199
482,WYNN,0.034918
376,QRVO,0.033473
302,MNK,0.032573
477,WMB,0.032224
389,RRC,0.032172
321,NFLX,0.031975


In [13]:
Top_10_lowest_yearly_volatilities

,Stock,Volatilities
259,KO,0.009063
87,CHD,0.009599
24,AJG,0.009711
352,PEP,0.009713
93,CLX,0.009725
142,DVA,0.009882
422,T,0.009943
468,VZ,0.009949
390,RSG,0.009973
355,PG,0.010106


### Question 2: Correlations

Analysts often care about the _correlation_ of stock prices between firms. Correlation measures the statistical similarity between the two prices' movements. If the prices move very similarly, the correlation of their _returns_  is close to 1. If they tend to make exactly the opposite movements (ie one price moves up and the other one down), the correlation is close to -1. If there is no clear statistical relationship between the movements of two stock prices, the correlation in their returns is close to zero.

For a sample of stock price returns $x,y$ with observations for $n$ days, the correlation $r_{xy}$ between $x$ and $y$ can be calculated as:

$$
r_{xy} = \frac{\sum x_i y_i - n \bar{x}\bar{y}}{ns_x s_y} = {\frac {n\sum x_{i}y_{i}-\sum x_{i}\sum y_{i}}{{\sqrt {n\sum x_{i}^{2}-(\sum x_{i})^{2}}}~{\sqrt {n\sum y_{i}^{2}-(\sum y_{i})^{2}}}}}.
$$

Here $\bar{x}$ refers to the average value of $x$ over the $n$ observations, and $s_x$ to its standard deviation.

Based on time series of the stock returns we just computed, we can calculate a  correlation value for each pair of stocks, for example between MSFT (Microsoft) and AAPL (Apple). This gives us a measure of the similarity between the two stocks in this time period.


Calculate all correlations between companies. You can search online for a `pandas` or `numpy` function that does this directly.

In [17]:
# Your code here
# calculate the correlation matrix of stock returns
pivoted_returns = data_unstack.pivot(index='Date', columns='Stock', values='Return')
correlation_matrix = pivoted_returns.corr()

### Question 2.1

Next, analyse the correlations between the companies:
- Define functions to print out the $n$ top and bottom correlated companies for any given company. 
- Use your functions to study the following companies in the tech sector: Amazon, Microsoft, Facebook, Apple, and Google. Comment on the results. Which (possibly other) companies are they most closely related to in terms of highest correlations? Would you have expected the results you see?

In [ ]:
# Your code here
def obtain_top_bottom(matrix, stock, top_n = 10):
    corr_for_stock = matrix[[stock]].reset_index()
    return corr_for_stock[corr_for_stock['Stock']!=stock].sort_values(stock, ascending=False).head(top_n), corr_for_stock[corr_for_stock['Stock']!=stock].sort_values(stock).head(top_n)

In [50]:
Top_10_highest_corr_AMZN, Top_10_lowest_corr_AMZN = obtain_top_bottom(correlation_matrix, 'AMZN')
Top_10_highest_corr_MSFT, Top_10_lowest_corr_MSFT = obtain_top_bottom(correlation_matrix, 'MSFT')
Top_10_highest_corr_FB, Top_10_lowest_corr_FB = obtain_top_bottom(correlation_matrix, 'FB')
Top_10_highest_corr_AAPL, Top_10_lowest_corr_AAPL = obtain_top_bottom(correlation_matrix, 'AAPL')
Top_10_highest_corr_GOOG, Top_10_lowest_corr_GOOG = obtain_top_bottom(correlation_matrix, 'GOOG')

## Part 2:  Clustering

**30% of grade**

In this part of the assignment, you will develop a clustering algorithm to study the similarity of different stocks. 

The general purpose of clustering analysis is dividing a set of objects into groups that are somehow "similar" to each other. It is a widespread tool used for exploratory data analysis in diverse fields in both science and business. For example, in marketing analytics, cluster analysis is employed to group consumers into segments based on their characteristics or _features_, such as age, post code, purchase history, etc. These features are somehow aggregated to compare the similarity between consumers. Based on this similarity, a clustering algorithm then divides the consumers into segments.

We will apply this idea on stock market data to identify groups of stocks that perform similarly over time. There are many reasons for grouping stocks together, such as analysing trading strategies, risk management, or simply presenting stock market information. Publicly traded companies are often grouped together by simple features such as the industry they operate in (eg tech companies or pharma companies), but here we'll take a data-driven approach, grouping together stocks that perform similarly over time. 

Cluster analysis is an umbrella term for many different algorithmic approaches. Here you'll develop one that's based on the concept of `greedy` algorithm design, specified below. You'll also have the opportunity to explore other approaches using Python libraries.

What is a good measure for stocks "performing similarly" to use for clustering. Let's use the one we just calculated: correlations in their returns. How can we use this similarity information for clustering? We now have access to all correlations between stock returns in S&P 500. We can think of this as a _graph_ as follows. The _nodes_ of the graph are the stocks (eg MSFT and AAPL). The _edges_ between them are the correlations, which we have just calculated between each stock, where the value of the correlation is the edge weight. Notice that since we have the correlations between all companies, this is a _dense_ graph, where all possible edges exist.

We thus have a graph representing pairwise "similarity" scores in correlations, and we want to divide the graph into clusters. There are many possible ways to do this, but here we'll use a _greedy_ algorithm design. The algorithm is as follows:

1. Sort the edges in the graph by their weight (ie the correlation), pick a number $k$ for the number of iterations of the algorithm
2. Create single-node sets from each node in the graph
3. Repeat $k$ times:
	1. Pick the graph edge with the highest correlation
	2. Combine the two sets containing the source and the destination of the edge
	3. Repeat with the next-highest weight edge
4. Return the remaining sets after the $k$ iterations 

What does the algorithm do? It first initializes a graph with no connections, where each node is in a separate set. Then in the main loop, it runs through the $k$ highest-weighted edges, and adds connections at those edges. This leads to sets being combined (or "merged"). The result is "groups" of stocks determined by the highest correlations between the stock returns. These are your stock clusters.

For example, suppose that the toy graph below represents four stocks: A,B,C,D and their return correlations. Suppose we pick $k=2$ and run the algorithm. 

<img src="cluster0.png" alt="cluster0" style="width: 200px;"/>


The algorithm would begin by initializing four separate sets of one node each: {A}, {B}, {C}, {D}. It would then first connect C and D because of their correlation 0.95, resulting in just three sets: {A}, {B}, and {C,D}. Then it would connect A and B, resulting in two sets of two nodes each: {A,B}, and {C,D}. These would be our clusters for $k=2$.

### Question 3: Implementing the algorithm

Your task is to implement the clustering algorithm using the functions below. First, for convenience in implementing the algorithm, let's create a list of the correlations from the pandas data. 

In [51]:
def create_correlation_list(correl):
    """
    Creates a list of correlations from a pandas dataframe of correlations
    
    Parameters:
        correl: pandas dataframe of correlations
    
    Returns:
        list of correlations containing tuples of form (correlation, ticker1, ticker2)
    """
    n_comp = len(correl.columns)
    comp_names = list(correl.columns)
    # Faster if we use a numpy matrix
    correl_mat = correl.to_numpy()
    L = [] # create list
    for i in range(n_comp):
        for j in range(i+1,n_comp):
            L.append((correl_mat[i,j],comp_names[i],comp_names[j]))
    return L

edges = create_correlation_list(correlation_matrix)

Next, let's turn to the algorithm itself. Consider the example above, repeated here.

<img src="cluster0.png" alt="cluster0" style="width: 200px;"/>


Suppose we pick $k=3$ and have sorted the edge list in step 1 of the algorithm. How should we represent the clusters in step 2? One great way is to use a dictionary where each _key_ is a node, and each _value_ is another node that this node "points to". A cluster is then a chain of these links, which we represent as a dictionary.

In step 2 of the algorithm, we start with four nodes that point to themselves, ie the dictionary `{'A':'A','B':'B','C':'C','D':'D'}`. When a node points to itself, it ends the chain. Here the clusters are thus just the nodes themselves, as in the figure below.

<img src="cluster1.png" alt="cluster1" style="width: 200px;"/>


Let's walk through the algorithm's next steps. We first look at the highest-weight edge, which is between C and D. These clusters will be combined. In terms of the dictionary, this means that one of them will not point to itself, but to the other one (here it does not matter which one). So we make the dictionary at `C` point to `D`. The dictionary becomes `{'A':'A','B':'B','C':'D','D':'D'}`.

<img src="cluster2.png" alt="cluster2" style="width: 200px;"/>


The next highest correlation is between A and B, so these clusters would be combined. The dictionary becomes `{'A':'B','B':'B','C':'D','D':'D'}`.

<img src="cluster3.png" alt="cluster3" style="width: 200px;"/>


The third highest correlation is between C and B. Let's think about combining these clusters using the dictionary we have. Looking up `B`, we get `B`: the node B is in the _bottom_ of the chain representing its cluster. But when we look up `C`, it points to `D`. Should we make `C` point to `B`? No - that would leave nothing  pointing at `D`, and `C` and `D` should remain connected! We could perhaps have `C` somehow point at both nodes, but that could become complicated, so we'll do the following instead. We'll follow the chain to the bottom. In the dictionary, we look up `C` and see that it points to `D`. We then look up `D` which points to itself, so `D` is the _bottom_ node. We then pick one of the bottom nodes `B` and `D`, and make it point to the other. We then have the dictionary `{'A':'B','B':'B','C':'D','D':'B'}`, and the corresponding clustering in the figure below.

<img src="cluster4.png" alt="cluster4" style="width: 200px;"/>


In other words, we'll keep track of clusters in a dictionary such that **each cluster has exactly one bottom node**. To do this, we need a mechanism for following a cluster to the bottom. You'll implement this in the function `find_bottom` below. The function takes as input a node and a dictionary, and runs through the "chain" in the dictionary until it finds a bottom node pointing to itself.

The other thing we'll need to do is combine clusters by connecting two nodes. This means taking the two nodes, finding the bottom node for each node's cluster, and making one point to the other. You'll implement this in the function `merge_clusters` below.

Finally, you'll need to set up the algorithm by sorting the correlations, and then looping through this merging $k$ times. You'll implement this in the function `cluster_correlations` below. This completes the algorithm.

But there is one more thing. If you only keep track of a dictionary like `{'A':'B','B':'B','C':'D','D':'B'}`, how do you actually find the clusters from the dictionary? A convenient way is to store some extra information: the "starting nodes" of each cluster to which no other node links. For example, above these "starting nodes" would include all nodes `A,B,C,D` in the beginning, but only `A` and `C` in the end. If we keep track of these, we can then write a function that starts from each such remaining "starting node", works through to the bottom, and creates the cluster along the way. You'll implement this in the function `construct_sets` below.

### Intermediary results

You can load a pre-computed set of results up to this point using the following commands.

In [53]:
# Load intermediary results from a "pickle" file
# You can use these with your algorithm below
import pickle
file_name = 'cluster_correlations'
with open(file_name, "rb") as f:
    correl = pickle.load(f)
    edges = pickle.load(f)

firms = list(correl.columns)
print(firms[:10])
edges[:10]

['MMM', 'ABT', 'ABBV', 'ACN', 'ATVI', 'AYI', 'ADBE', 'AAP', 'AES', 'AET']


[(np.float64(0.598666164029738), 'MMM', 'ABT'),
 (np.float64(0.32263699601940254), 'MMM', 'ABBV'),
 (np.float64(0.6320593488560189), 'MMM', 'ACN'),
 (np.float64(0.41855006701119907), 'MMM', 'ATVI'),
 (np.float64(0.4508974957132859), 'MMM', 'AYI'),
 (np.float64(0.4687548443045165), 'MMM', 'ADBE'),
 (np.float64(0.25713165217544326), 'MMM', 'AAP'),
 (np.float64(0.33537796741224424), 'MMM', 'AES'),
 (np.float64(0.31737374099675925), 'MMM', 'AET'),
 (np.float64(0.5059306055816828), 'MMM', 'AMG')]

### Clustering implementation

Complete the following functions to implement the clustering algorithm.

In [ ]:
def find_bottom(node, next_nodes):
    """
    Find the "bottom" of a cluster starting from node in dictionary next_nodes

    Parameters:
        node: starting node
        next_nodes: dictionary of node connections

    Returns:
        the bottom node in the cluster
    """
    # Your code here
    
    pass


def merge_sets(node1, node2, next_nodes, set_starters):
    """
    Merges the clusters containing node1, node2 using the connections dictionary next_nodes.
    Also removes any bottom node which is no longer a "starting node" from set_starters.

    Parameters:
        node1: first node the set of which will be merged
        node2: second node the set of which will be merged
        next_nodes: dictionary of node connections
        set_starters: set of nodes that "start" a cluster

    Returns:
        does this function need to return something?
    """
    # Your code here


def cluster_correlations(edge_list, firms, k=200):
    """
    A mystery clustering algorithm
     
    Parameters:
         edge_list - list of edges of the form (weight,source,destination)
         firms - list of firms (tickers)
         k - number of iterations of algorithm

    Returns:
         next_nodes - dictionary to store clusters as "pointers"
            - the dictionary keys are the nodes and the values are the node in the same cluster that the key node points to
         set_starters - set of nodes that no other node points to (this will be used to construct the sets below)

    Algorithm:
         1 sort edges by weight (highest correlation first)
         2 initialize next_nodes so that each node points to itself (single-node clusters)
         3 take highest correlation edge
            check if the source and destination are in the same cluster using find_bottom
            if not, merge the source and destination nodes' clusters using merge_sets
         4 if max iterations not reached, repeat 3 with next highest correlation
         (meanwhile also keep track of the "set starters" ie nodes that have nothing pointing to them for later use)
    """
    # Sort edges
    sorted_edges = _____
    # Initialize dictionary of pointers
    next_nodes = {node: node for node in firms}
    # Keep track of "starting nodes", ie nodes that no other node points to in next_nodes
    set_starters = {node for node in firms}

    # Loop k times
    for i in range(k):
        # Your algorithm here
        
    return set_starters, next_nodes

Once we've run the algorithm, we'll need to construct the clusters. You can use the function below to do so.

In [ ]:
def construct_sets(set_starters, next_nodes):
    """
    Constructs sets (clusters) from the next_nodes dictionary
    
    Parameters:
        set_starters: set of starting nodes 
        next_nodes: dictionary of connections
    
    Returns: 
        dictionary of sets (clusters):
            key - bottom node of set; value - set of all nodes in the cluster
    
    """
    # Initialise an empty dictionary 
    all_sets = dict()
    
    # Loop:
    # Start from each set starter node
    # Construct a "current set" with all nodes on the way to bottom node
    # If bottom node is already a key of all_sets, combine the "current set" with the one in all_sets,
    # Otherwise add "current set" to all_sets
    for s in set_starters:
        cur_set = set()
        cur_set.add(s)
        p = s
        while next_nodes[p] != p:
            p = next_nodes[p]
            cur_set.add(p)
            
        if p not in all_sets:
            all_sets[p] = cur_set
        else: 
            for item in cur_set:
                all_sets[p].add(item)
    return all_sets

all_clusters = construct_sets(set_starters,next_nodes)

### Question 3.2: analysing the results

After you have implemented the algorithm in Python, add cells below answering the following questions:
- Do some detective work: what is the algorithm that you've implemented called? In what other graph problem is it often used? How are the problems related? (Hint: the algorithm is mentioned on the Wikipedia page for greedy algorithms.)
- Run the algorithm and present the results formatted in a useful way. 
- Discuss the results for different values of $k$.  
- Do the resulting clusters "make sense"? (You may need to search online what the companies do.) Verify that the stocks in your clusters perform similarly by plotting the returns and the (normalised) stock prices for some of the clusters.
- You may use graphs etc. to present your results.

## Part 3: 

**40% of grade**

Depending on your interests, you may work on either subsection below, or both. You might go deeper into one question than another, but for an outstanding grade, you should have at least some discussion on both.

You may use generative AI such as chatGPT to help with this part of the assignment (but not the other parts).

### In-depth analysis

The project is _open_ in the sense that you can probably think of further interesting questions to look into based on returns, correlations, and clusters. This is not required but being creative and going further than the above questions will make your work stand out. You can explore one or several of the ideas below, or come up with questions of your own.

Depending on your interests, you might look at different things. For example, when researching the algorithm, you might be interested in its complexity, and how to improve your implementation's efficiency. On Wikipedia, you may find a couple of ways to drastically improve the algorithm speed, but are relatively small changes to your code.

If you're more interested in the financial applications of clustering, there are also opportunities to think about further steps. For example, some people claim that you can derive trading strategies based on clustering - that often one of the stocks in a cluster is a leader and the others follow that price. If this is true, you could track the price of the leader stock and then trade the other stocks in the cluster based on changes in the leader's price. Do you think this would make sense? Do you have an idea on how to identify a leader stock?

You might also want to repeat the analysis for different time periods. You would be able to do this by looking at the code for the second homework to figure out how to read data from Yahoo Finance using pandas, and going through the process for all companies in the csv file for another time period. Perhaps you could explore for example how correlations between companies have changed over time, or how clusters found by your algorithm change over time.

### Exploring other clustering methods

You've used just one approach to clustering, and arguably not the best one. Research clustering algorithms and libraries to apply them in Python. Discuss some other algorithms that could be used, and how they differ from the one you've implemented. Look at the Python library `scikit-learn`. How would you apply the clustering algorithms provided by the library to stock price data? Would you need to develop new metrics other than correlations? If you want to go even further,  try running some of these other clustering algorithms on your data, and report the results. Start from here: http://scikit-learn.org/stable/modules/clustering.html#clustering; you'll find a stock market example there too. For future reference, you may also find other interesting machine-learning tools for both stock market analysis or other analytics purposes.

### Question 4

Create cells below to add your extra part as code and narrative text explaining your idea and results.

## All done!

Make sure to follow submission guidelines.